In [1]:
# ตรวจสอบ GPU
!nvidia-smi

Tue Feb 10 00:38:21 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 591.59                 Driver Version: 591.59         CUDA Version: 13.1     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5060 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P5              9W /  180W |    1960MiB /  16311MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Download Library

---

In [2]:
# pip install gdown matplotlib seaborn pandas numpy scikit-learn tensorflow statsmodels

In [3]:
# !pip install -U scikit-learn==1.4.2

# Libraries

---

In [4]:
import numpy as np
import pandas as pd

from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error
)

import gdown
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet


# Load Data

---

In [5]:
# # ใช้ครั้งแรกเพื่อโหลดไฟล์

# gdown.download(
#     id="1_Sxdx7lMn7RAsY-eE6kljGk8x39nJkon",
#     output="TestScenarioPred.csv",
#     quiet=False
# )

# gdown.download(
#     id="1-bNSbMCJvcxVsxvsiEHaPHIQBlLiFQgu",
#     output="divorce_all_model.csv",
#     quiet=False
# )


In [6]:
# โหลดข้อมูล (ทำ IQR มาแล้ว)
df = pd.read_csv("divorce_all_model.csv")

In [7]:
df["ds"] = pd.to_datetime(df["ds"])
df = df.sort_values("ds")
df_prophet = df.copy()
df_prophet = df_prophet.rename(columns={"Divorce": "y"})
df_prophet["ds"] = pd.to_datetime(df_prophet["ds"])
df_prophet = df_prophet.set_index("ds")
df_prophet = df_prophet.sort_index()
df_arima = df.copy()
df_arima = df_arima.set_index("ds")
df_arima = df_arima.sort_index()
df = df.rename(columns={"y": "Divorce"})

In [8]:
df

,ds,Divorce,Marriage
0,2007-01-01,6568.0,20912.0
1,2007-02-01,5963.0,31475.0
2,2007-03-01,6388.0,20155.0
3,2007-04-01,6696.0,21182.0
4,2007-05-01,6298.0,21678.0
...,...,...,...
211,2024-08-01,7929.0,12443.0
212,2024-09-01,7618.0,12239.0
213,2024-10-01,8618.0,12055.0
214,2024-11-01,7953.0,12858.0


In [9]:
df_arima

,Divorce,Marriage
ds,,
2007-01-01,6568.0,20912.0
2007-02-01,5963.0,31475.0
2007-03-01,6388.0,20155.0
2007-04-01,6696.0,21182.0
2007-05-01,6298.0,21678.0
...,...,...
2024-08-01,7929.0,12443.0
2024-09-01,7618.0,12239.0
2024-10-01,8618.0,12055.0


In [10]:
df

,ds,Divorce,Marriage
0,2007-01-01,6568.0,20912.0
1,2007-02-01,5963.0,31475.0
2,2007-03-01,6388.0,20155.0
3,2007-04-01,6696.0,21182.0
4,2007-05-01,6298.0,21678.0
...,...,...,...
211,2024-08-01,7929.0,12443.0
212,2024-09-01,7618.0,12239.0
213,2024-10-01,8618.0,12055.0
214,2024-11-01,7953.0,12858.0


In [11]:
# ข้อมูลมีทั้งหมด 216 บรรทัด 2 แถว.
# Col ที่ 1    "ds"     มีข้อมูล 216 บรรทัด ไม่มีค่า null เป็นประเภท Datetime
# Col ที่ 2  "Divorce"  มีข้อมูล 216 บรรทัด ไม่มีค่า null เป็นประเภท Float
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ds        216 non-null    datetime64[ns]
 1   Divorce   216 non-null    float64       
 2   Marriage  216 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 5.2 KB


In [12]:
# ข้อมูลนับได้ทั้งหมด 216 บรรทัดและมี 2 แถว
# ข้อมูลที่นำมาใช้ มีตั้งแต่ปี พ.ศ. 2550 - 2567
#                     ค.ศ. 2007 - 2024
# หย่าร้างกันต่ำสุดอยู่ที่ 3275.0 คู่
# หย่าร้างกันสูงสุดอยู่ที่ 9007.0 คู่
# คิดเป็นค่าเฉลี่ยจากผลรวมทั้งหมดคือ 7093.98 คู่ จากจำนวนทั้งหมด
# เบี่ยงเบนมาตราฐาน(std) มีการกระจายตัวอยู่ราว ๆ 691.20 คู่
#

df.describe()

,ds,Divorce,Marriage
count,216,216.000000,216.000000
mean,2015-12-16 10:53:20,7093.976852,18013.125000
min,2007-01-01 00:00:00,3275.000000,6845.000000
25%,2011-06-23 12:00:00,6638.750000,15788.000000
50%,2015-12-16 12:00:00,7067.000000,17246.000000
75%,2020-06-08 12:00:00,7565.500000,19333.250000
max,2024-12-01 00:00:00,9007.000000,33313.000000
std,NaN,691.204096,4089.292655


In [13]:
fig = px.line(
    df.reset_index(), # <------------------------- ดึง Row มาดูเลย ตั้งแต่ปี 2021 - 2023
    x="ds",
    y="Divorce",
    title="Raw data"
)

fig.update_layout(
    template="plotly_white",
    hovermode="x unified"
)

fig.update_xaxes(
    dtick="M12",               # 👈 ปรับเป็น "M1" เพื่อให้โชว์ทุกเดือน หรือ "M3" สำหรับทุกไตรมาส
    # tickformat="%b %Y",       # 👈 %b คือชื่อเดือนย่อ (เช่น Jan, Feb) %Y คือปี
    tickformat="%Y",
    showgrid=True,
    tickangle=-45,            # 👈 เอียงเล็กน้อยเพื่อให้ชื่อเดือนไม่ซ้อนกัน
    hoverformat="%Y"    # 👈 Hover ให้เห็นละเอียดถึงระดับวัน/เดือน/ปี
)

fig.show()

# Prophet

---

นำค่า Best Parameter จากการ Hyperparameter Tuning มาใช้

In [14]:
params = {
    'changepoint_prior_scale': 1,
    'seasonality_prior_scale': 0.1,
    'yearly_seasonality': True,
    'weekly_seasonality': False,
    'daily_seasonality': False
}

กำหนดค่า Caps และ Floor สำหรับการทำ Future forecast ของ Prophet

In [15]:
df_prophet_final = df_prophet.reset_index()

cap_value = df_prophet_final["y"].max() * 1.2 # กำหนดให้การเติบโตจะไม่เกิน 20%
df_prophet_final["cap"] = cap_value
df_prophet_final["floor"] = 0                 # และจะไม่ติดลบ โดยตั้งให้ต่ำสุดจะอยู่ที่ 0


In [16]:
df_prophet_final

,ds,y,Marriage,cap,floor
0,2007-01-01,6568.0,20912.0,10808.4,0
1,2007-02-01,5963.0,31475.0,10808.4,0
2,2007-03-01,6388.0,20155.0,10808.4,0
3,2007-04-01,6696.0,21182.0,10808.4,0
4,2007-05-01,6298.0,21678.0,10808.4,0
...,...,...,...,...,...
211,2024-08-01,7929.0,12443.0,10808.4,0
212,2024-09-01,7618.0,12239.0,10808.4,0
213,2024-10-01,8618.0,12055.0,10808.4,0
214,2024-11-01,7953.0,12858.0,10808.4,0


https://facebook.github.io/prophet/docs/saturating_forecasts.html

ใช้ Logistic Growth ของ Prophet ในการทำ Cap / Floor

In [17]:
m_final = Prophet(
    growth='logistic',
    **params
)

m_final.fit(df_prophet_final)


00:38:24 - cmdstanpy - INFO - Chain [1] start processing
00:38:24 - cmdstanpy - INFO - Chain [1] done processing


In [18]:
future = m_final.make_future_dataframe(periods=60, freq="MS")
future["cap"] = cap_value
future["floor"] = 0

forecast_future = m_final.predict(future)


In [19]:
fig_future = go.Figure()

fig_future.add_trace(
    go.Scatter(
        x=df_prophet.index,
        y=df_prophet["y"],
        name="Actual",
        line=dict(color="black", width=2)
    )
)

fig_future.add_trace(
    go.Scatter(
        x=forecast_future["ds"],
        y=forecast_future["yhat"],
        name="Future Forecast",
        line=dict(color="red", dash="dash")
    )
)

fig_future.add_trace(
    go.Scatter(
        x=forecast_future["ds"],
        y=forecast_future["yhat_upper"],
        line=dict(width=0),
        showlegend=False
    )
)

fig_future.add_trace(
    go.Scatter(
        x=forecast_future["ds"],
        y=forecast_future["yhat_lower"],
        fill="tonexty",
        fillcolor="rgba(255,0,0,0.15)",
        line=dict(width=0),
        name="Confidence Interval"
    )
)

fig_future.update_layout(
    title="Future Forecast of Divorce Cases (Prophet – Tuned Model)",
    xaxis_title="Date",
    yaxis_title="Number of Divorces",
    hovermode="x unified",
    template="plotly_white",
    height=600
)

fig_future.show()


In [ ]:
metric_df = (
    forecast_future[['ds', 'yhat']]
    .merge(df_prophet_final[['ds', 'y']], on='ds', how='inner')
)

y_true = metric_df['y']
y_pred = metric_df['yhat']

p_mae = mean_absolute_error(y_true, y_pred)
p_mse = mean_squared_error(y_true, y_pred)
p_rmse = np.sqrt(p_mse)
p_mape = mean_absolute_percentage_error(y_true, y_pred) * 100

print(f"MAE : {p_mae:.2f}")
print(f"RMSE: {p_rmse:.2f}")
print(f"MAPE: {p_mape:.2f}%")


MAE : 287.10
RMSE: 452.38
MAPE: 4.34%


In [56]:
df

,ds,Divorce,Marriage
0,2007-01-01,6568.0,20912.0
1,2007-02-01,5963.0,31475.0
2,2007-03-01,6388.0,20155.0
3,2007-04-01,6696.0,21182.0
4,2007-05-01,6298.0,21678.0
...,...,...,...
211,2024-08-01,7929.0,12443.0
212,2024-09-01,7618.0,12239.0
213,2024-10-01,8618.0,12055.0
214,2024-11-01,7953.0,12858.0


In [58]:
# ===============================
# Merge y (actual) from df_prophet_final
# ===============================
forecast_merged = (
    forecast_future
    .merge(
        df_prophet_final[["ds", "y"]],
        on="ds",
        how="left"
    )
)

# ===============================
# Select + Rename columns
# ===============================
forecast_export = (
    forecast_merged[[
        "ds",
        "y",
        "yhat",
        "cap",
        "floor",
        "yhat_lower",
        "yhat_upper",
        "trend_lower",
        "trend_upper"
    ]]
    .rename(columns={
        "y": "Divorce",
        "yhat": "Forecast"
    })
)

# ===============================
# Save to CSV
# ===============================
forecast_export.to_csv("prophet_forecast_future.csv", index=False)


# SARIMA

---

เริ่มทำ SARIMA

In [20]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

กำหนดค่า Parameter

In [21]:
pdq = (1, 1, 2)
PDQL = (1, 1, 1, 12) # Time Steps 12 เดือน

In [49]:
def expanding_sarimax_full_monthly(
    y,
    pdq,
    PDQL,
    train_years,
    test_years
):
    summary_rows = []
    forecast_rows = []
    arima_model_results = {}

    train_months = train_years * 12
    test_months  = test_years * 12

    y = y.sort_index()

    train_start = y.index.min()
    train_end   = train_start + pd.DateOffset(months=train_months - 1)

    round_no = 1

    # =========================
    # Rolling / Expanding (Evaluation)
    # =========================
    while True:

        test_start = train_end + pd.DateOffset(months=1)
        test_end   = test_start + pd.DateOffset(months=test_months - 1)

        if test_end > y.index.max():
            break

        y_train = y.loc[train_start:train_end]
        y_test  = y.loc[test_start:test_end]

        if len(y_test) != test_months:
            break

        model = SARIMAX(
            y_train,
            order=pdq,
            seasonal_order=PDQL,
            enforce_stationarity=False,
            enforce_invertibility=False
        )

        fitted = model.fit(disp=False)

        forecast = fitted.forecast(steps=test_months)
        forecast.index = y_test.index

        mae  = mean_absolute_error(y_test, forecast)
        rmse = np.sqrt(mean_squared_error(y_test, forecast))
        mape = mean_absolute_percentage_error(y_test, forecast) * 100

        round_name = f"Round_{round_no}"

        summary_rows.append({
            "Round": round_name,
            "Train_Start": train_start,
            "Train_End": train_end,
            "Test_Start": test_start,
            "Test_End": test_end,
            "Train_Months": len(y_train),
            "Test_Months": len(y_test),
            "MAE": mae,
            "RMSE": rmse,
            "MAPE": mape
        })

        arima_model_results[round_name] = fitted

        for ds, yhat in forecast.items():
            forecast_rows.append({
                "Round": round_name,
                "Test_Year": ds.year,
                "ds": ds,
                "forecast": yhat
            })

        print(
            f'✔ {round_name} | '
            f'Train: {train_start.date()} → {train_end.date()} | '
            f'Test: {test_start.date()} → {test_end.date()} | '
            f'RMSE={rmse:.2f}'
        )

        train_end += pd.DateOffset(months=test_months)
        round_no += 1

    # =========================
    # ✅ FINAL MODEL (Train with ALL data)
    # =========================
    final_model = SARIMAX(
        y,
        order=pdq,
        seasonal_order=PDQL,
        enforce_stationarity=False,
        enforce_invertibility=False
    )

    final_fitted = final_model.fit(disp=False)

    arima_model_results["Final_Full_Train"] = final_fitted

    print(
        f'✅ Final model trained with full data '
        f'({y.index.min().date()} → {y.index.max().date()})'
    )

    arima_summary_df  = pd.DataFrame(summary_rows)
    arima_forecast_df = pd.DataFrame(forecast_rows)

    return arima_summary_df, arima_forecast_df, arima_model_results


In [23]:
df_arima

,Divorce,Marriage
ds,,
2007-01-01,6568.0,20912.0
2007-02-01,5963.0,31475.0
2007-03-01,6388.0,20155.0
2007-04-01,6696.0,21182.0
2007-05-01,6298.0,21678.0
...,...,...
2024-08-01,7929.0,12443.0
2024-09-01,7618.0,12239.0
2024-10-01,8618.0,12055.0


In [50]:
arima_summary_df, arima_forecast_df, arima_model_results = (
    expanding_sarimax_full_monthly(
        y=df_arima["Divorce"],
        pdq=pdq,
        PDQL=PDQL,
        train_years=2,
        test_years=1
    )
)


C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site

✔ Round_1 | Train: 2007-01-01 → 2008-12-01 | Test: 2009-01-01 → 2009-12-01 | RMSE=379.08
✔ Round_2 | Train: 2007-01-01 → 2009-12-01 | Test: 2010-01-01 → 2010-12-01 | RMSE=377.42


C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site

✔ Round_3 | Train: 2007-01-01 → 2010-12-01 | Test: 2011-01-01 → 2011-12-01 | RMSE=322.96
✔ Round_4 | Train: 2007-01-01 → 2011-12-01 | Test: 2012-01-01 → 2012-12-01 | RMSE=421.69


C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



...

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



✔ Round_16 | Train: 2007-01-01 → 2023-12-01 | Test: 2024-01-01 → 2024-12-01 | RMSE=371.04


C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



✅ Final model trained with full data (2007-01-01 → 2024-12-01)


C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [51]:
arima_summary_df


,Round,Train_Start,Train_End,Test_Start,Test_End,Train_Months,Test_Months,MAE,RMSE,MAPE
0,Round_1,2007-01-01,2008-12-01,2009-01-01,2009-12-01,24,12,305.511656,379.079278,4.640966
1,Round_2,2007-01-01,2009-12-01,2010-01-01,2010-12-01,36,12,320.705541,377.417226,4.806150
2,Round_3,2007-01-01,2010-12-01,2011-01-01,2011-12-01,48,12,248.921201,322.958928,3.883563
3,Round_4,2007-01-01,2011-12-01,2012-01-01,2012-12-01,60,12,331.348150,421.691183,4.872839
4,Round_5,2007-01-01,2012-12-01,2013-01-01,2013-12-01,72,12,276.374676,354.263666,4.376719
5,Round_6,2007-01-01,2013-12-01,2014-01-01,2014-12-01,84,12,349.231798,409.356083,5.044792
6,Round_7,2007-01-01,2014-12-01,2015-01-01,2015-12-01,96,12,302.496615,350.002679,4.244888
7,Round_8,2007-01-01,2015-12-01,2016-01-01,2016-12-01,108,12,239.967364,322.706148,3.336110
8,Round_9,2007-01-01,2016-12-01,2017-01-01,2017-12-01,120,12,207.272340,250.738544,2.898586
9,Round_10,2007-01-01,2017-12-01,2018-01-01,2018-12-01,132,12,243.617084,282.957009,3.353658


In [52]:
arima_forecast_df

,Round,Test_Year,ds,forecast
0,Round_1,2009,2009-01-01,7028.171766
1,Round_1,2009,2009-02-01,7074.201284
2,Round_1,2009,2009-03-01,7157.691563
3,Round_1,2009,2009-04-01,7189.520925
4,Round_1,2009,2009-05-01,7121.097952
...,...,...,...,...
187,Round_16,2024,2024-08-01,7635.346121
188,Round_16,2024,2024-09-01,7650.811415
189,Round_16,2024,2024-10-01,7918.359196
190,Round_16,2024,2024-11-01,7644.316156


In [27]:
# ==============================
# SARIMAX → Forecast table
# ==============================
arima_forecast_std = arima_forecast_df.copy()

# arima_forecast_std = arima_forecast_std.rename(columns={
#     "forecast": "Forecast"
# })

# join Actual จากข้อมูลจริง
actual_map = df_arima["Divorce"].rename("Actual")

arima_forecast_std = (
    arima_forecast_std
    .merge(
        actual_map,
        left_on="ds",
        right_index=True,
        how="left"
    )
)

arima_forecast_std["Model"] = "SARIMAX"

arima_forecast_std = arima_forecast_std[
    ["Model", "Round", "ds", "Actual", "forecast"]
]


In [28]:
def shorten_train_test(df):
    df = df.copy()

    # normalize column names
    df.columns = df.columns.str.upper()

    # safety check
    required_cols = [
        "ROUND",
        "TRAIN_START", "TRAIN_END",
        "TEST_START", "TEST_END",
        "MAE", "RMSE", "MAPE"
    ]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    # convert to datetime
    for col in ["TRAIN_START", "TRAIN_END", "TEST_START", "TEST_END"]:
        df[col] = pd.to_datetime(df[col])

    # create short labels (เหมือน Prophet)
    df["TRAIN"] = (
        df["TRAIN_START"].dt.year.astype(str)
        + "–"
        + df["TRAIN_END"].dt.year.astype(str)
    )

    df["TEST"] = (
        df["TEST_START"].dt.year.astype(str)
        + "–"
        + df["TEST_END"].dt.year.astype(str)
    )

    # final schema for comparison
    final_cols = [
        "ROUND",
        "TRAIN",
        "TEST",
        "MAE",
        "RMSE",
        "MAPE"
    ]

    return df[final_cols]


In [29]:
arima_summary_df = shorten_train_test(arima_summary_df)

In [30]:
arima_forecast_std

,Model,Round,ds,Actual,forecast
0,SARIMAX,Round_1,2009-01-01,6910.0,7028.171766
1,SARIMAX,Round_1,2009-02-01,6504.0,7074.201284
2,SARIMAX,Round_1,2009-03-01,7068.0,7157.691563
3,SARIMAX,Round_1,2009-04-01,6644.0,7189.520925
4,SARIMAX,Round_1,2009-05-01,6546.0,7121.097952
...,...,...,...,...,...
187,SARIMAX,Round_16,2024-08-01,7929.0,7635.346121
188,SARIMAX,Round_16,2024-09-01,7618.0,7650.811415
189,SARIMAX,Round_16,2024-10-01,8618.0,7918.359196
190,SARIMAX,Round_16,2024-11-01,7953.0,7644.316156


In [31]:
arima_summary_df

,ROUND,TRAIN,TEST,MAE,RMSE,MAPE
0,Round_1,2007–2008,2009–2009,305.511656,379.079278,4.640966
1,Round_2,2007–2009,2010–2010,320.705541,377.417226,4.806150
2,Round_3,2007–2010,2011–2011,248.921201,322.958928,3.883563
3,Round_4,2007–2011,2012–2012,331.348150,421.691183,4.872839
4,Round_5,2007–2012,2013–2013,276.374676,354.263666,4.376719
5,Round_6,2007–2013,2014–2014,349.231798,409.356083,5.044792
6,Round_7,2007–2014,2015–2015,302.496615,350.002679,4.244888
7,Round_8,2007–2015,2016–2016,239.967364,322.706148,3.336110
8,Round_9,2007–2016,2017–2017,207.272340,250.738544,2.898586
9,Round_10,2007–2017,2018–2018,243.617084,282.957009,3.353658


In [32]:
arima_summary_df[["MAE", "RMSE", "MAPE"]].mean()


MAE     367.855264
RMSE    475.579307
MAPE      5.521323
dtype: float64

In [33]:
fig = go.Figure()

# Actual
fig.add_trace(go.Scatter(
    x=df_arima.index,
    y=df_arima["Divorce"],
    mode="lines",
    name="Actual",
    line=dict(width=2)
))

# Forecast per Test Year
for year in sorted(arima_forecast_df["Test_Year"].unique()):
    sub = arima_forecast_df[arima_forecast_df["Test_Year"] == year]

    fig.add_trace(go.Scatter(
        x=sub["ds"],
        y=sub["forecast"],
        mode="lines",
        name=f"Forecast {year}",
        line=dict(dash="dash")
    ))

fig.update_layout(
    title="Expanding Window SARIMAX Forecast",
    xaxis_title="Date",
    yaxis_title="Divorce",
    hovermode="x unified",
    height=600
)

fig.show()


In [34]:
import numpy as np
import pandas as pd

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error
)

# -------------------------
# 1) Fit SARIMAX
# -------------------------
sarimax_model = SARIMAX(
    df_arima["Divorce"],
    order=pdq,
    seasonal_order=PDQL,
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarimax_fit = sarimax_model.fit(disp=False)

# =====================================================
# 2) 🔢 In-sample prediction (ไว้คำนวณ MAE RMSE MAPE)
# =====================================================
insample_pred = sarimax_fit.get_prediction(
    start=df_arima.index[0],
    end=df_arima.index[-1],
    dynamic=False
)

y_true = df_arima["Divorce"]
y_pred = insample_pred.predicted_mean

# -------------------------
# 3) Metrics
# -------------------------
mae  = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = mean_absolute_percentage_error(y_true, y_pred) * 100

sarimax_metrics = pd.DataFrame({
    "Model": ["SARIMAX"],
    "MAE": [mae],
    "RMSE": [rmse],
    "MAPE": [mape]
})

sarimax_metrics


C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\nanta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



,Model,MAE,RMSE,MAPE
0,SARIMAX,369.962713,700.709279,5.598205


In [35]:
# -------------------------
# 4) จำนวนเดือนอนาคต
# -------------------------
N_FUTURE_MONTHS = 60  # 5 ปี

# -------------------------
# 5) Forecast อนาคต + CI
# -------------------------
future_res = sarimax_fit.get_forecast(steps=N_FUTURE_MONTHS)

yhat = future_res.predicted_mean
conf_int = future_res.conf_int(alpha=0.05)  # 95% CI

# -------------------------
# 6) สร้าง index รายเดือน
# -------------------------
future_index = pd.date_range(
    start=df_arima.index[-1] + pd.DateOffset(months=1),
    periods=N_FUTURE_MONTHS,
    freq="MS"
)

yhat.index = future_index
conf_int.index = future_index

# -------------------------
# 7) รวมเป็น DataFrame (มี Cap / Floor)
# -------------------------
sarimax_future_df = pd.DataFrame({
    "ds": future_index,
    "yhat": yhat.values,
    "floor": conf_int.iloc[:, 0].values,
    "cap": conf_int.iloc[:, 1].values
})

sarimax_future_df.head()


,ds,yhat,floor,cap
0,2025-01-01,8174.989240,7210.522548,9139.455932
1,2025-02-01,7734.807290,6697.568415,8772.046164
2,2025-03-01,8257.911142,7212.500629,9303.321655
3,2025-04-01,8096.927661,7049.521849,9144.333473
4,2025-05-01,8138.196868,7089.934395,9186.459341


In [36]:
import plotly.graph_objects as go

fig_future = go.Figure()

# -------------------------
# 1) Actual data
# -------------------------
fig_future.add_trace(go.Scatter(
    x=df_arima.index,
    y=df_arima["Divorce"],
    mode="lines",
    name="Actual",
    line=dict(width=2)
))

# -------------------------
# 2) Future forecast (yhat)
# -------------------------
fig_future.add_trace(go.Scatter(
    x=sarimax_future_df["ds"],
    y=sarimax_future_df["yhat"], #<------------------------------------------------- Future forecast
    mode="lines",
    name="SARIMAX Forecast",
    line=dict(width=3, dash="dot")
))

# -------------------------
# 3) Confidence interval (Cap / Floor)
# -------------------------
# Floor
fig_future.add_trace(go.Scatter(
    x=sarimax_future_df["ds"],
    y=sarimax_future_df["floor"],
    mode="lines",
    line=dict(width=0),
    showlegend=False,
    hoverinfo="skip"
))

# Cap (fill to floor)
fig_future.add_trace(go.Scatter(
    x=sarimax_future_df["ds"],
    y=sarimax_future_df["cap"], 
    mode="lines",
    fill="tonexty",
    fillcolor="rgba(0, 0, 0, 0.15)",
    line=dict(width=0),
    name="Confidence Interval (95%)",
    hoverinfo="skip"
))

# -------------------------
# 4) Vertical line: End of observed data
# -------------------------
last_obs_date = df_arima.index[-1]

fig_future.add_shape(
    type="line",
    x0=last_obs_date,
    x1=last_obs_date,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(
        color="gray",
        dash="dash",
        width=2
    )
)

fig_future.add_annotation(
    x=last_obs_date,
    y=1,
    xref="x",
    yref="paper",
    text="End of Observed Data",
    showarrow=False,
    yanchor="bottom",
    font=dict(color="gray")
)

# -------------------------
# 5) Layout
# -------------------------
fig_future.update_layout(
    title="SARIMAX Future Forecast (5 Years)",
    xaxis_title="Date",
    yaxis_title="Divorce",
    hovermode="x unified",
    height=600
)

fig_future.show()


# Compare models

---

In [46]:
# ===============================
# Prophet – Native Output
# ===============================
print(f"MAE : {p_mae:.2f}")
print(f"RMSE: {p_rmse:.2f}")
print(f"MAPE: {p_mape:.2f}%")


MAE : 287.10
RMSE: 452.38
MAPE: 4.34%


In [43]:
# ===============================
# SARIMAX – Rolling metrics (native)
# ===============================
arima_summary_df
arima_summary_df[["MAE", "RMSE", "MAPE"]].mean()

MAE     367.855264
RMSE    475.579307
MAPE      5.521323
dtype: float64

In [47]:
FUTURE_STEPS = 60
fig_future_all = go.Figure()

# Actual
fig_future_all.add_trace(go.Scatter(
    x=df["ds"],
    y=df["Divorce"],
    name="Actual",
    line=dict(color="black", width=2)
))


# Prophet
fig_future_all.add_trace(go.Scatter(
    x=forecast_future["ds"],
    y=forecast_future["yhat"],
    name="Prophet Forecast",
    line=dict(dash="dot")
))

# SARIMAX (ใช้ model สุดท้าย)
last_arima_model = list(arima_model_results.values())[-1]
sarimax_future = last_arima_model.forecast(steps=FUTURE_STEPS)

fig_future_all.add_trace(go.Scatter(
    x=sarimax_future.index,
    y=sarimax_future.values,
    name="SARIMAX Forecast",
    line=dict(dash="dashdot")
))

fig_future_all.update_layout(
    title="Future Forecast Comparison (Prophet vs SARIMAX)",
    xaxis_title="Date",
    yaxis_title="Divorce",
    hovermode="x unified",
    height=600
)

fig_future_all.show()


In [ ]:
arima_forecast_std

,Model,Round,ds,Actual,forecast
0,SARIMAX,Round_1,2010-01-01,6983.0,7559.883845
1,SARIMAX,Round_1,2010-02-01,6314.0,6890.015231
2,SARIMAX,Round_1,2010-03-01,7036.0,7273.843328
3,SARIMAX,Round_1,2010-04-01,7342.0,6865.937090
4,SARIMAX,Round_1,2010-05-01,6909.0,6936.058695
...,...,...,...,...,...
175,SARIMAX,Round_15,2024-08-01,7929.0,7635.346121
176,SARIMAX,Round_15,2024-09-01,7618.0,7650.811415
177,SARIMAX,Round_15,2024-10-01,8618.0,7918.359196
178,SARIMAX,Round_15,2024-11-01,7953.0,7644.316156


In [53]:
fig_roll_all = go.Figure()

# Actual
fig_roll_all.add_trace(go.Scatter(
    x=df["ds"],
    y=df["Divorce"],
    name="Actual",
    line=dict(color="black", width=2)
))


# Prophet Rolling
fig_roll_all.add_trace(go.Scatter(
    x=forecast_future["ds"],
    y=forecast_future["yhat"],
    name="ProphetForecast",
    mode="lines"
))

# SARIMAX Rolling
fig_roll_all.add_trace(go.Scatter(
    x=arima_forecast_std["ds"],
    y=arima_forecast_std["forecast"],
    name="SARIMAX Rolling Forecast",
    mode="lines"
))

fig_roll_all.update_layout(
    title="Forecast Comparison Prophet vs SARIMAX",
    xaxis_title="Date",
    yaxis_title="Divorce",
    hovermode="x unified",
    height=600
)

fig_roll_all.show()


In [ ]:
arima_forecast_std

,Model,Round,ds,Actual,forecast
0,SARIMAX,Round_1,2010-01-01,6983.0,7559.883845
1,SARIMAX,Round_1,2010-02-01,6314.0,6890.015231
2,SARIMAX,Round_1,2010-03-01,7036.0,7273.843328
3,SARIMAX,Round_1,2010-04-01,7342.0,6865.937090
4,SARIMAX,Round_1,2010-05-01,6909.0,6936.058695
...,...,...,...,...,...
175,SARIMAX,Round_15,2024-08-01,7929.0,7635.346121
176,SARIMAX,Round_15,2024-09-01,7618.0,7650.811415
177,SARIMAX,Round_15,2024-10-01,8618.0,7918.359196
178,SARIMAX,Round_15,2024-11-01,7953.0,7644.316156


In [ ]:
arima_summary_df.to_csv("sarimax_metrics.csv", index=False)

arima_forecast_std.to_csv("sarimax_rolling_forecast.csv", index=False)

sarimax_future_df.to_csv("sarimax_future.csv", index=False)

In [ ]:
arima_summary_df

,ROUND,TRAIN,TEST,MAE,RMSE,MAPE
0,Round_1,2007–2009,2010–2010,320.705541,377.417226,4.806150
1,Round_2,2007–2010,2011–2011,248.921201,322.958928,3.883563
2,Round_3,2007–2011,2012–2012,331.348150,421.691183,4.872839
3,Round_4,2007–2012,2013–2013,276.374676,354.263666,4.376719
4,Round_5,2007–2013,2014–2014,349.231798,409.356083,5.044792
5,Round_6,2007–2014,2015–2015,302.496615,350.002679,4.244888
6,Round_7,2007–2015,2016–2016,239.967364,322.706148,3.336110
7,Round_8,2007–2016,2017–2017,207.272340,250.738544,2.898586
8,Round_9,2007–2017,2018–2018,243.617084,282.957009,3.353658
9,Round_10,2007–2018,2019–2019,178.534948,207.941661,2.445345


In [ ]:
arima_forecast_std

,Model,Round,ds,Actual,forecast
0,SARIMAX,Round_1,2010-01-01,6983.0,7559.883845
1,SARIMAX,Round_1,2010-02-01,6314.0,6890.015231
2,SARIMAX,Round_1,2010-03-01,7036.0,7273.843328
3,SARIMAX,Round_1,2010-04-01,7342.0,6865.937090
4,SARIMAX,Round_1,2010-05-01,6909.0,6936.058695
...,...,...,...,...,...
175,SARIMAX,Round_15,2024-08-01,7929.0,7635.346121
176,SARIMAX,Round_15,2024-09-01,7618.0,7650.811415
177,SARIMAX,Round_15,2024-10-01,8618.0,7918.359196
178,SARIMAX,Round_15,2024-11-01,7953.0,7644.316156


In [ ]:
forecast_future

,ds,trend,cap,floor,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2007-01-01,6635.486696,10808.4,0,6287.201495,7552.514923,6635.486696,6635.486696,286.053911,286.053911,286.053911,286.053911,286.053911,286.053911,0.0,0.0,0.0,6921.540607
1,2007-02-01,6640.076852,10808.4,0,5789.230076,7069.881409,6640.076852,6640.076852,-187.936580,-187.936580,-187.936580,-187.936580,-187.936580,-187.936580,0.0,0.0,0.0,6452.140272
2,2007-03-01,6644.221182,10808.4,0,6244.535013,7555.279936,6644.221182,6644.221182,266.730625,266.730625,266.730625,266.730625,266.730625,266.730625,0.0,0.0,0.0,6910.951807
3,2007-04-01,6648.807753,10808.4,0,6087.372360,7375.605052,6648.807753,6648.807753,114.259946,114.259946,114.259946,114.259946,114.259946,114.259946,0.0,0.0,0.0,6763.067699
4,2007-05-01,6653.244568,10808.4,0,6163.188719,7489.121418,6653.244568,6653.244568,148.900720,148.900720,148.900720,148.900720,148.900720,148.900720,0.0,0.0,0.0,6802.145287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,2029-08-01,7786.192678,10808.4,0,6988.646313,8307.472967,7785.913314,7786.442098,-144.419654,-144.419654,-144.419654,-144.419654,-144.419654,-144.419654,0.0,0.0,0.0,7641.773024
272,2029-09-01,7790.142478,10808.4,0,6947.634279,8304.961851,7789.855643,7790.399992,-153.560425,-153.560425,-153.560425,-153.560425,-153.560425,-153.560425,0.0,0.0,0.0,7636.582053
273,2029-10-01,7793.961854,10808.4,0,7251.878554,8556.757597,7793.670002,7794.228853,106.063636,106.063636,106.063636,106.063636,106.063636,106.063636,0.0,0.0,0.0,7900.025489
274,2029-11-01,7797.905426,10808.4,0,6984.623988,8307.989296,7797.608523,7798.183317,-162.218303,-162.218303,-162.218303,-162.218303,-162.218303,-162.218303,0.0,0.0,0.0,7635.687123


In [ ]:
sarimax_future_df

,ds,yhat,floor,cap
0,2025-01-01,8174.989240,7210.522548,9139.455932
1,2025-02-01,7734.807290,6697.568415,8772.046164
2,2025-03-01,8257.911142,7212.500629,9303.321655
3,2025-04-01,8096.927661,7049.521849,9144.333473
4,2025-05-01,8138.196868,7089.934395,9186.459341
5,2025-06-01,8049.759565,7000.898922,9098.620208
6,2025-07-01,7724.377090,6674.985282,8773.768898
7,2025-08-01,7801.866804,6751.962266,8851.771342
8,2025-09-01,7787.159225,6736.747123,8837.571326
9,2025-10-01,8091.532534,7040.614090,9142.450978


# Scenario Data Scene

---

ดึงข้อมูลจากไฟล์ csv มาเพื่อทำ Scenario Analysis ของ Prophet

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import plotly.graph_objects as go

# ---------------------------------------------------------
# 1. Load Data
# ---------------------------------------------------------

# ข้อมูลสำหรับสร้าง Simulation Logic (2021-2024)
df_scenario_base = pd.read_csv('TestScenarioPred.csv')
if not pd.api.types.is_datetime64_any_dtype(df_scenario_base['ds']):
    df_scenario_base['ds'] = pd.to_datetime(df_scenario_base['ds'])
df_scenario_base

,ds,y,Marriage
0,2021-01-01,7416.0,16879.0
1,2021-02-01,7288.0,22997.0
2,2021-03-01,9063.0,20298.0
3,2021-04-01,7913.0,16614.0
4,2021-05-01,7023.0,15319.0
5,2021-06-01,7363.0,15523.0
6,2021-07-01,6098.0,13904.0
7,2021-08-01,5800.0,12622.0
8,2021-09-01,6685.0,14001.0
9,2021-10-01,7276.0,13866.0


สร้าง Function เพื่อคำนวณ Mean, Std, Floor, Cap แยกตามเดือน

In [ ]:
# =========================================================
# Function 1: หา Cap/Floor และค่าเฉลี่ยในแต่ละเดือน (Seasonal Stats)
# =========================================================
def get_monthly_stats(df_scenario):
    """
    รับ DataFrame (เช่นข้อมูลช่วง 2021–2024)
    คำนวณ Mean, Std, Floor, Cap แยกตามเดือน
    """
    df = df_scenario.copy()

    # สร้าง month ถ้ายังไม่มี
    if "month" not in df.columns:
        df["month"] = df["ds"].dt.month

    monthly_stats = (
        df.groupby("month")["y"]
        .agg(
            avg="mean",
            std="std",
            floor_min=lambda x: x.quantile(0.2),
            cap_max=lambda x: x.quantile(0.8)
        )
        .reset_index()
    )

    return monthly_stats

print(get_monthly_stats(df_scenario_base))

    month      avg          std  floor_min  cap_max
0       1  8407.75   680.057044     8076.0   8843.4
1       2  7747.25   394.224450     7488.4   8002.4
2       3  8652.50   640.890266     8194.0   9145.8
3       4  8393.75   321.214752     8283.2   8562.8
4       5  8299.00   948.804511     7730.4   8936.4
5       6  8093.50   539.024118     7822.0   8398.0
6       7  7398.50   909.692072     6912.8   8002.4
7       8  7592.75  1232.961847     7077.4   8262.2
8       9  8118.75  1330.984692     7419.4   8747.4
9      10  8701.50  1230.827499     7939.0   9441.4
10     11  8309.75   304.765894     8123.0   8509.4
11     12  8269.50   297.278209     8019.2   8525.2


สร้าง Function สำหรับสร้างวันที่ปลอม เพื่อนำมาทำ Scenario Data

In [ ]:
# =========================================================
# Function 2: สุ่มค่า 12 ค่าสำหรับทุกปี (Simulation)
# =========================================================
def generate_random_scenario(monthly_stats, start_date, sim_years):
    """
    รับค่า Stats รายเดือน, วันที่เริ่มต้น, และจำนวนปีที่ต้องการสุ่ม (sim_years)
    ทำการสุ่มตัวเลขโดยอิงจาก Mean/Std และคุมด้วย Min/Max ของเดือนนั้นๆ
    """
    future_rows = []
    
    # สร้าง Loop ตามจำนวนปี
    for i in range(1, sim_years + 1):
        # สร้าง Loop 12 เดือน
        for month in range(1, 13):
            # ดึงค่าสถิติของเดือนนั้นๆ
            stats = monthly_stats[monthly_stats['month'] == month].iloc[0]
            
            # 1. Random ด้วย NumPy (Normal Distribution)
            # ใช้ random.normal(mean, std)
            rand_val = np.random.normal(stats['avg'], stats['std'])
            
            # 2. Clip ค่าให้อยู่ไม่เกิน Cap/Floor ของเดือนนั้น (MaxMin)
            final_val = np.clip(rand_val, stats['floor_min'], stats['cap_max'])
            
            # คำนวณวันที่
            # (คำนวณแบบง่าย: บวกเดือนเพิ่มไปเรื่อยๆ หรือสร้าง date object ใหม่)
            # เพื่อความแม่นยำใช้ pd.DateOffset
            current_date = start_date + pd.DateOffset(years=(i-1), months=month)
            
            future_rows.append({
                'ds': current_date,
                'y': final_val
            })
            
    return pd.DataFrame(future_rows)

กำหนดจำนวนปีที่จะสร้าง

In [ ]:
sim_year = 5  # <----------------------------------------------------------------------- User กำหนดจำนวนปีตรงนี้

ทำการสร้าง Scenario Data

In [ ]:
# =========================================================
# ส่วนการนำไปใช้งาน (Execution)
# =========================================================

# 1. Load Data Scenario (2021-2024) อยู่ข้างบน
# df_scenario_base = pd.read_csv('TestScenarioPred.csv')
# if not pd.api.types.is_datetime64_any_dtype(df_scenario_base['ds']):
#     df_scenario_base['ds'] = pd.to_datetime(df_scenario_base['ds'])

# 2. เรียกใช้ Function 1: หา Stats รายเดือน
month_stats = get_monthly_stats(df_scenario_base)
# print(month_stats) # ดูค่าที่ได้

# 3. User Input & เรียกใช้ Function 2: สุ่มค่า
last_date_history = df_prophet.index.max() # วันที่สุดท้ายของข้อมูลจริง

df_simulated = generate_random_scenario(
    monthly_stats=month_stats, 
    start_date=last_date_history, 
    sim_years=sim_year
)

# 4. รวมข้อมูล (Data Augmentation)
# นำ df_prophet (ข้อมูลจริง) มารวมกับ df_simulated (ข้อมูลสุ่ม)
# df_prophet_reset = df_prophet.reset_index()
df_augmented = pd.concat([df_scenario_base[['ds', 'y']], df_simulated], ignore_index=True)

# 5. กำหนด Global Cap/Floor สำหรับ Prophet (Logistic Growth)
# ต้องใส่ให้ครบทุกแถวทั้งเก่าและใหม่
global_cap = df_augmented['y'].max() * 1
global_floor = 0

df_augmented['cap'] = global_cap
df_augmented['floor'] = global_floor

# 6. Train Prophet Model
m_final = Prophet(
    growth="logistic",
    **best_params # parameter ที่จูนมาแล้ว
)

m_final.fit(df_augmented)

# 7. Predict (ดู Trend บนข้อมูลที่รวมแล้ว)
# สร้าง Future เท่ากับความยาวข้อมูล (ไม่ต้องเพิ่ม periods เพราะเรา append ข้อมูลสุ่มไปแล้ว)
future = m_final.make_future_dataframe(periods=60, freq='MS')
future['cap'] = global_cap
future['floor'] = global_floor

forecast_future = m_final.predict(future)


21:06:25 - cmdstanpy - INFO - Chain [1] start processing
21:06:25 - cmdstanpy - INFO - Chain [1] done processing


Plot ข้อมูลหลังจากทำมาดู

In [ ]:
# 8. Plot Graph (Plotly)
fig_future = go.Figure()

# เส้นข้อมูลจริง (แก้ไข: ใช้ df_scenario_base แทน df_prophet_reset ที่เอาออกไป)
fig_future.add_trace(go.Scatter(
    x=df_scenario_base['ds'], y=df_scenario_base['y'],
    name="Actual History", line=dict(color='black')
))

# จุดข้อมูลที่สุ่มขึ้นมา (Simulation) (แก้ไข: ใส่เป็นเส้น + จุดกากบาท x)
fig_future.add_trace(go.Scatter(
    x=df_simulated['ds'], y=df_simulated['y'],
    name=f"Simulated Data ({sim_year} Years)",
    mode='lines+markers', 
    marker=dict(symbol='x', color='orange', size=8, opacity=0.8),
    line=dict(color='orange', width=1)
))

# เส้น Trend ที่ Prophet ทำนายได้
fig_future.add_trace(go.Scatter(
    x=forecast_future['ds'], y=forecast_future['yhat'],
    name="Scenario Trend", line=dict(color='red', dash='dash')
))

# Confidence Interval
fig_future.add_trace(go.Scatter(
    x=forecast_future['ds'], y=forecast_future['yhat_upper'],
    line=dict(width=0), showlegend=False
))
fig_future.add_trace(go.Scatter(
    x=forecast_future['ds'], y=forecast_future['yhat_lower'],
    fill='tonexty', fillcolor='rgba(255,0,0,0.15)',
    line=dict(width=0), name="Confidence Interval"
))

fig_future.update_layout(title=f"Scenario Simulation: {sim_year} Years", hovermode="x unified")
fig_future.show()

In [ ]:
# ไม่จำเป็น
# forecast_future.to_csv("Ramdom_y_prophet.csv",
#                         index=False,
#                         encoding="utf-8-sig"
#                         )

In [ ]:
# เช็คจำนวนปีทั้งหมดเฉย ๆ 
howmany = (2550,2551,2552,2553,2554,2555,2556,2557,2558,2559,2560,2561,2562,2563,2564,2565,2566,2567)
len(howmany)

18